In [1]:
import os
import numpy as np
import json
from detectron2.structures import BoxMode
import cv2
from detectron2.utils.visualizer import Visualizer


In [2]:
def get_collembola_dicts(img_dir):
    json_file = os.path.join(img_dir, "train.json")
    with open(json_file) as f:
        imgs_anns = json.load(f)

    dataset_dicts = []
    for idx, v in enumerate(imgs_anns.values()):
        record = {}
        
        filename = os.path.join(img_dir, v["filename"])
        height = 3464
        width = 5202
        
        record["file_name"] = filename
        record["image_id"] = idx
        record["height"] = height
        record["width"] = width
      
        annos = v["regions"]
        objs = []
        for _, anno in annos.items():
            assert not anno["region_attributes"]
            anno = anno["shape_attributes"]
            px = anno["all_points_x"]
            py = anno["all_points_y"]
            poly = [(x + 0.5, y + 0.5) for x, y in zip(px, py)]
            poly = [p for x in poly for p in x]

            obj = {
                "bbox": [np.min(px), np.min(py), np.max(px), np.max(py)],
                "bbox_mode": BoxMode.XYXY_ABS,
                "segmentation": [poly],
                "category_id": 0,
            }
            objs.append(obj)
        record["annotations"] = objs
        dataset_dicts.append(record)
    return dataset_dicts

from detectron2.data import DatasetCatalog, MetadataCatalog
for d in ["train", "val"]:
    DatasetCatalog.register("balloon_" + d, lambda d=d: get_balloon_dicts("balloon/" + d))
    MetadataCatalog.get("balloon_" + d).set(thing_classes=["balloon"])
balloon_metadata = MetadataCatalog.get("balloon_train")

In [3]:
img_dir = "train"
json_file = "train.json"
json_file = os.path.join(img_dir, json_file)
with open(json_file) as f:
    imgs_anns = json.load(f)


In [4]:
from detectron2.data.datasets import register_coco_instances
register_coco_instances("train", {}, "train/train.json", "train")
register_coco_instances("test", {}, "test/test.json", "test")

In [5]:
import random
import cv2
from detectron2.utils.visualizer import Visualizer


dataset_dicts = DatasetCatalog.get("train")
dataset_metadata = MetadataCatalog.get("train")

for d in random.sample(dataset_dicts, 3):
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=dataset_metadata, scale=0.5)
    vis = visualizer.draw_dataset_dict(d)
    cv2.imwrite("test2.jpg",vis.get_image()[:, :, ::-1])


Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.



In [6]:
from detectron2.engine import DefaultTrainer
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog


cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("train",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.OUTPUT_DIR = "/home/stephan/Desktop/collembola_ai/8000"
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
cfg.SOLVER.MAX_ITER = 8000 
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 10
cfg.nms = True

In [36]:
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

[08/19 17:06:25 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

Unable to load 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (11, 1024) in the model!
Unable to load 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (11,) in the model!
Unable to load 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (40, 1024) in the model!
Unable to load 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (40,) in the model!


[08/19 17:06:25 d2.engine.train_loop]: Starting training from iteration 0
[08/19 17:06:35 d2.utils.events]:  eta: 0:57:55  iter: 19  total_loss: 3.239  loss_cls: 2.470  loss_box_reg: 0.613  loss_rpn_cls: 0.107  loss_rpn_loc: 0.049  time: 0.4629  data_time: 0.2943  lr: 0.000005  max_mem: 2679M
[08/19 17:06:44 d2.utils.events]:  eta: 1:00:28  iter: 39  total_loss: 3.085  loss_cls: 2.312  loss_box_reg: 0.623  loss_rpn_cls: 0.059  loss_rpn_loc: 0.040  time: 0.4665  data_time: 0.2867  lr: 0.000010  max_mem: 2679M
[08/19 17:06:54 d2.utils.events]:  eta: 1:00:19  iter: 59  total_loss: 2.784  loss_cls: 2.070  loss_box_reg: 0.612  loss_rpn_cls: 0.032  loss_rpn_loc: 0.031  time: 0.4683  data_time: 0.2928  lr: 0.000015  max_mem: 2679M
[08/19 17:07:02 d2.utils.events]:  eta: 0:59:15  iter: 79  total_loss: 2.521  loss_cls: 1.745  loss_box_reg: 0.674  loss_rpn_cls: 0.023  loss_rpn_loc: 0.036  time: 0.4616  data_time: 0.2652  lr: 0.000020  max_mem: 2679M
[08/19 17:07:12 d2.utils.events]:  eta: 0:59:2

In [8]:
# Look at training curves in tensorboard:
!kill 16886
%load_ext tensorboard
%tensorboard --logdir output

/usr/bin/sh: 1: kill: No such process



In [18]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.55 # set the testing threshold for this model
cfg.DATASETS.TEST = ("test", )
predictor = DefaultPredictor(cfg)

In [8]:
from detectron2.utils.visualizer import ColorMode
dataset_dicts_test = DatasetCatalog.get("test")
dataset_metadata_test = MetadataCatalog.get("test")



Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.



In [57]:
appendix = "4000"
output_folder = "annotated_test_" + appendix
output_folder2 = "test_set2_" + appendix
!mkdir $output_folder
i = 0
for d in dataset_dicts_test:
    output_name = output_folder + "/annotated_" + str(i) + ".jpg"
    img = cv2.imread(d["file_name"])
    outputs = predictor(img)
    visualizer = Visualizer(img[:, :, ::-1], metadata=dataset_metadata_test, scale=1.)
    instances = outputs["instances"].to("cpu")
    vis = visualizer.draw_instance_predictions(instances)
    result = vis.get_image()[:, :, ::-1]
    write_res = cv2.imwrite(output_name, result)
    i += 1

%cd $output_folder
!mkdir $output_folder2
%cd ..
for i in os.listdir("test_set2"):
    if ".jpg" in i:
        file_path = os.path.join("test_set2",i)
        im = cv2.imread(file_path)
        outputs = predictor(im)
        v = Visualizer(
            im[:, :, ::-1],
            metadata=dataset_metadata,
            scale=1.,
            instance_mode=ColorMode.SEGMENTATION 
        )
        instances = outputs["instances"].to("cpu")
        v = v.draw_instance_predictions(instances)
        result = v.get_image()[:, :, ::-1]
        output_name = output_folder + "/" + output_folder2 + "/annotated_" + str(i) + ".jpg"
        write_res = cv2.imwrite(output_name, result)

mkdir: cannot create directory ‘annotated_test_4000’: File exists


'%cd $output_folder\n!mkdir $output_folder2\n%cd ..\nfor i in os.listdir("test_set2"):\n    if ".jpg" in i:\n        file_path = os.path.join("test_set2",i)\n        im = cv2.imread(file_path)\n        outputs = predictor(im)\n        v = Visualizer(\n            im[:, :, ::-1],\n            metadata=dataset_metadata,\n            scale=1.,\n            instance_mode=ColorMode.SEGMENTATION \n        )\n        instances = outputs["instances"].to("cpu")\n        v = v.draw_instance_predictions(instances)\n        result = v.get_image()[:, :, ::-1]\n        output_name = output_folder + "/" + output_folder2 + "/annotated_" + str(i) + ".jpg"\n        write_res = cv2.imwrite(output_name, result)'

In [9]:
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

In [19]:
#single image prediction
output_name = "mix_plate4_pred.jpg"
img = cv2.imread("mix_plate4.jpg")
outputs = predictor(img)
visualizer = Visualizer(img[:, :, ::-1], metadata=dataset_metadata_test, scale=1.)
instances = outputs["instances"].to("cpu")
vis = visualizer.draw_instance_predictions(instances)
result = vis.get_image()[:, :, ::-1]
write_res = cv2.imwrite(output_name, result)

In [23]:
outputs

{'instances': Instances(num_instances=3, image_height=3464, image_width=5202, fields=[pred_boxes: Boxes(tensor([[2485.3262, 1451.4491, 2678.9089, 1606.8192],
         [   9.7335, 1183.7520,  220.7334, 1346.5566],
         [1284.9249,   72.8346, 1391.6996,  192.3983]], device='cuda:0')), scores: tensor([0.8165, 0.6698, 0.5815], device='cuda:0'), pred_classes: tensor([9, 9, 4], device='cuda:0')])}

In [41]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
evaluator = COCOEvaluator("test", cfg, False, output_dir="./output/")
val_loader = build_detection_test_loader(cfg, "test")
inference_on_dataset(trainer.model, val_loader, evaluator)

WARNING [07/20 16:12:49 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[07/20 16:12:49 d2.data.datasets.coco]: Loaded 48 images in COCO format from test/test.json
[07/20 16:12:49 d2.data.common]: Serializing 48 elements to byte tensors and concatenating them all ...
[07/20 16:12:49 d2.data.common]: Serialized dataset takes 0.02 MiB
[07/20 16:12:49 d2.evaluation.evaluator]: Start inference on 48 images
[07/20 16:12:52 d2.evaluation.evaluator]: Inference done 11/48. 0.0739 s / img. ETA=0:00:09
[07/20 16:12:58 d2.evaluation.evaluator]: Inference done 31/48. 0.0719 s / img. ETA=0:00:04
[07/20 16:13:02 d2.evaluation.evaluator]: Total inference time: 0:00:10.755284 (0.250123 s / img per device, on 1 devices)
[07/20 16:13:02 d2.evaluation.evaluator]: Total inference pure compute time: 0:00:03 (0.071709 s / img per device, on 1 devices)
[07/20 16:13:02 d2.evaluation.coco_evaluation]: Preparing results for COCO format ...
[07/20

OrderedDict([('bbox',
              {'AP': 61.82127631506105,
               'AP50': 89.44090331742582,
               'AP75': 75.5029113291549,
               'APs': nan,
               'APm': 64.99999999999999,
               'APl': 61.88797164210822,
               'AP-ceratophysella': 74.72772277227723,
               'AP-desoria': 72.9664586421086,
               'AP-deuterosminthurus': 69.35055696469392,
               'AP-folsomia': 51.054035988770266,
               'AP-megalothorax': 46.01726754083555,
               'AP-pseudosinella': 77.86340818955844,
               'AP-sinella': 63.836546435885126,
               'AP-sminthurides': 57.109076371302606,
               'AP-species01': 57.10389586553677,
               'AP-sphaeridia': 48.18379437964183})])

In [35]:
#single picture
os.makedirs("annotated_results", exist_ok=True)

file_path = "mix.jpg"
im = cv2.imread(file_path)
outputs = predictor(im)
v = Visualizer(
    im[:, :, ::-1],
    metadata=dataset_metadata_test,
    scale=1.,
    instance_mode=ColorMode.IMAGE
)
instances = outputs["instances"].to("cpu")
#instances.remove('pred_masks')
v = v.draw_instance_predictions(instances)
result = v.get_image()[:, :, ::-1]
file_name = "mix_2.jpg"
write_res = cv2.imwrite(f'annotated_results/{file_name}', result)

In [18]:
for i in os.listdir("test"):
    if ".jpg" in i:
        file_path = os.path.join("test",i)
        im = cv2.imread(file_path)
        outputs = predictor(im)
        v = Visualizer(
            im[:, :, ::-1],
            metadata=dataset_metadata_test,
            scale=1.,
            instance_mode=ColorMode.IMAGE
        )
        instances = outputs["instances"].to("cpu")
        #instances.remove('pred_masks')
        v = v.draw_instance_predictions(instances)
        result = v.get_image()[:, :, ::-1]
        file_name = i
        write_res = cv2.imwrite(f'annotated_results/{file_name}', result)

KeyboardInterrupt: 

/home/stephan/Desktop/collembola_ai/annotated_results
/home/stephan/Desktop/collembola_ai
